# Day034 - Joseph P. Merten
## Data Analytics 2: Data Visualization & Predictions - Day 034 notes
- Following along with this video: https://www.youtube.com/watch?v=xi0vhXFPegw

### Wednesday  practice technical interview.

### MSSQL is not required,

# Start of code along...

### Imports...

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')    #    Set style sheet
pd.set_option('display.max_columns', 300)    #    Set the number of columns displayed



from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn import tree 
from sklearn.tree import DecisionTreeClassifier

In [2]:
cancer_df = pd.read_csv("./cancer.csv")

In [3]:
cancer_df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883
...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,NaN,0.05533
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648
567,927241,M,20.60,29.33,140.10,1265.0,NaN,0.27700,0.35140,0.15200,0.2397,0.07016


In [4]:
print(cancer_df.describe())
print(cancer_df.info())
# cancer_df.shape()

                 id  radius_mean  texture_mean  perimeter_mean    area_mean  \
count  5.690000e+02   569.000000    569.000000      569.000000   569.000000   
mean   3.037183e+07    14.127292     19.289649       91.969033   654.889104   
std    1.250206e+08     3.524049      4.301036       24.298981   351.914129   
min    8.670000e+03     6.981000      9.710000       43.790000   143.500000   
25%    8.692180e+05    11.700000     16.170000       75.170000   420.300000   
50%    9.060240e+05    13.370000     18.840000       86.240000   551.100000   
75%    8.813129e+06    15.780000     21.800000      104.100000   782.700000   
max    9.113205e+08    28.110000     39.280000      188.500000  2501.000000   

       smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
count       565.000000        569.000000      569.000000           569.000000   
mean          0.096336          0.104341        0.088799             0.048919   
std           0.014029          0.052813     

In [5]:
cancer_df['diagnosis'].value_counts(0, dropna=False)

B    357
M    212
Name: diagnosis, dtype: int64

### Can we use the columns (observations) in the df to predict malignant/benign?  True/False outcome.
1. Diagnosis only has two values T/F There is no missing data.
1. All other variables except ID can be candidates.

### Compute 1 or zero to flag is cancer is present...

In [6]:
cancer_df.loc[cancer_df['diagnosis']=='M', 'cancer_present'] = 1
cancer_df.loc[cancer_df['diagnosis']=='B', 'cancer_present'] = 0

### Compute 1 or zero to flag is cancer is present (Alternate approach)...

In [7]:
cancer_df['cancer_present'] = cancer_df['diagnosis'].apply(
    lambda x: np.nan if pd.isnull(x) else
    1 if x == 'M' else
    0 if x == 'B' else
    -1)
cancer_df['cancer_present'].value_counts(0, dropna=False)

0    357
1    212
Name: cancer_present, dtype: int64

### separate the features we're gonig to use as input.
1. The remaining columns are numereic.
1. Check for missing values.

In [8]:
cancer_df.isnull().sum()

id                        0
diagnosis                 0
radius_mean               0
texture_mean              0
perimeter_mean            0
area_mean                 0
smoothness_mean           4
compactness_mean          0
concavity_mean            0
concave points_mean       0
symmetry_mean             5
fractal_dimension_mean    6
cancer_present            0
dtype: int64

### Split the data into target variable (diagnosis) and the input features.
1. X - input matrix of values to use for prediction
1. y = pandas series/dataframe with the answer.

In [9]:
#  Use the rest of the features as input
# cancer_df.columns
input_features = list(cancer_df.columns[2:])
# print(input_features)
X = cancer_df[input_features]
y = cancer_df['diagnosis']
print(type(X), type(y))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


#  Horizontlly split the data into training and validatrion rows.
1. classification
1. regression
1. ???

Look into parameters for scikit-learn

In [10]:
from sklearn.model_selection import train_test_split
cancer_X_train, cancer_X_test, cancer_y_train, cancer_y_test = train_test_split(X, y, test_size=0.25, random_state=0)  #  25% test size
print(cancer_X_train.shape, cancer_X_test.shape, cancer_y_train.shape, cancer_y_test.shape)

(426, 11) (143, 11) (426,) (143,)


### We want the scale of the columns to be similar, otherwise it may have a larger impact on the prediction.  This is called normalization. Build a model pipeline.
1. fix missing values.
1. normalize the different columjs/features such that the transformed features are roughly on the same scale.  This will make ML's job easier.

In [11]:
tree1 = Pipeline([
    ('imp_mean', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('scaler', StandardScaler()),
    ('tree', DecisionTreeClassifier(criterion='entropy', random_state=0))
])
    
tree1.fit(cancer_X_train, cancer_y_train)

Pipeline(steps=[('imp_mean', SimpleImputer()), ('scaler', StandardScaler()),
                ('tree',
                 DecisionTreeClassifier(criterion='entropy', random_state=0))])

### Now evaluate the pipeline/model performance. using cross validation.

In [13]:
perf_scores = cross_val_score(tree1, cancer_X_train, cancer_y_train, cv=10)
# perf_scores is an array.  Clos to 1.0 indicates perfect, 0.00 is garbage.  Alll values of 1.0 is suspicious and indicates model over-fitting. (memorizing inputs)
print(perf_scores)
#  Our problem here is that [cancer_present] is included in our training data.

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


# Practice Interview 
- Practice writing a class with constructor and properties
- data types
- correlation
- finding missing values
- noSQL - focus on EDA & Predictive analysis.
- Search youtube for practice interview questions.

### Get notes from the video
https://grandcircus.zoom.us/rec/share/-5PcKvHjjHCuqZJ3gK6EL5_8ic4G0qRUieWCQfihCFftipKC8i0DFygtrB2rx5sm.Vi_-rksBcOWTKPz8
Passcode: pB38@FT%


### Google Colab URL:
https://colab.research.google.com/drive/1bhN-QB6Mp5ttnx6Req-OqZJ0HUlPEA9h#scrollTo=Sq1Xomc_tWLV

### 05/18/2023 - recap colab...

In [1]:
### if we sns.plot the dataset can show linear relatinoships in the data.

###  train_test_split() separates a percentage of data for training.  The remaining data will be used for cross validation.
### We need to address two issues:
- Missing data
- normalize the scale of ther data.

Look deeper into imp_mean, scaler, tree in the Pipeline() function.

### tree1.score(x_test, y_test) tests accuracy of the model.  If everything is 10% accurater - it should raise a red flag.  Are we overfitting?
- we wnt the model to predict, not memorize.
- max_depth limits the tree depth.  
-- Does it increase performnance?  
-- What result does it return?
- look at confusion matrix - wiki link in Jason's colab.
- There's a function n sklearn
### Confusion matrix:
https://en.wikipedia.org/wiki/Confusion_matrix
- accuracy = 
- precision = tp/(tp+fp)
- recall = tp/(tp+fn)
Should not judge models imbalanced data set on accuracy!
### accuracy_score from sklearn gives us precision, recall, F1 and support.

### Permutation importance.
### Feature importance - using different metrics.
- If the relative importance matches, they will probably be a good candidate,


In [5]:
for i in range(24, 32):
    print(i, i*9/5+32)

24 75.2
25 77.0
26 78.8
27 80.6
28 82.4
29 84.2
30 86.0
31 87.8
